You will need to run the following code in order to use this notebook:

`conda install -c h2oai h2o`

https://github.com/h2oai/h2o-tutorials

## Preprocessing

In [1]:
#Import the models we will be using
import h2o
# Import H2O Grid Search:
from h2o.grid.grid_search import H2OGridSearch

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.181-b13, mixed mode)


C:\Users\zgeorge\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\backend\server.py:369: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Starting server from C:\Users\zgeorge\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\zgeorge\AppData\Local\Temp\tmp6w6t84au
  JVM stdout: C:\Users\zgeorge\AppData\Local\Temp\tmp6w6t84au\h2o_zgeorge_started_from_python.out
  JVM stderr: C:\Users\zgeorge\AppData\Local\Temp\tmp6w6t84au\h2o_zgeorge_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/Denver
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,11 days
H2O cluster name:,H2O_from_python_zgeorge_hyy8rr
H2O cluster total nodes:,1
H2O cluster free memory:,247.5 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [3]:
#data = h2o.import_file(r"C:\Users\zgeorge\Dropbox (CMN Hospitals)\George\School\Fall 2018\Applied Machine Learning\AML_code\Final Project\Buy More Data.csv")

funds_csv = r"C:\Users\zgeorge\Dropbox (CMN Hospitals)\George\School\Fall 2018\Applied Machine Learning\AML_code\Final Project\Buy More Data.csv"  # modify this for your machine
data = h2o.import_file(funds_csv)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
data.shape

(21353, 28)

In [5]:
data['target'] = data['target'].asfactor()  #encode the binary repsonse as a factor
data['target'].levels()  #optional: after encoding, this shows the two factor levels, '0' and '1'

[['0', '1']]

In [6]:
# Partition data into 70%, 15%, 15% chunks
# Setting a seed will guarantee reproducibility

splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]

In [7]:
print(train.nrow)
print(valid.nrow)
print(test.nrow)

15010
3202
3141


In [8]:
y = 'target'
x = list(data.columns)

In [9]:
x.remove(y)  #remove the response
x.remove('change')  #remove the interest rate column because it's correlated with the outcome

In [10]:
# List of predictor columns
x

['Locationid',
 'Location',
 'LocPropertyType',
 'LocProvince',
 'LocPostal',
 'LocDistrict',
 'LocRegion',
 'RegionName',
 'Market',
 'PopulationRank',
 'FundraisingYear',
 'cy_funds',
 'cy_1_funds',
 'cy_2_funds',
 'cy_checkins',
 'cy_1_checkins',
 'cy_2_checkins',
 'cy_contacts',
 'cy_1_contacts',
 'cy_2_contacts',
 'cy_images',
 'cy_1_images',
 'cy_2_images',
 'cy_notes',
 'cy_1_notes',
 'cy_2_notes']

### Linear Regression

In [11]:
# Import H2O GLM:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [12]:
# Initialize the GLM estimator:
# Similar to R's glm() and H2O's R GLM, H2O's GLM has the "family" argument

glm_fit1 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit1')

In [13]:
glm_fit1.train(x=x, y=y, training_frame=train)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [14]:
glm_fit2 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit2', lambda_search=True)
glm_fit2.train(x=x, y=y, training_frame=train, validation_frame=valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [15]:
glm_perf1 = glm_fit1.model_performance(test)
glm_perf2 = glm_fit2.model_performance(test)

In [16]:
# Print model performance
print(glm_perf1)
print(glm_perf2)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.018162687706667974
RMSE: 0.13476901612265327
LogLoss: 0.07452178906817303
Null degrees of freedom: 3140
Residual degrees of freedom: 3087
Null deviance: 3147.763150497247
Residual deviance: 468.1458789262629
AIC: 576.1458789262629
AUC: 0.9996310238676293
pr_auc: 0.8332830482084576
Gini: 0.9992620477352585
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.43921408008400464: 


,0,1,Error,Rate
0,2501.0,11.0,0.0044,(11.0/2512.0)
1,9.0,620.0,0.0143,(9.0/629.0)
Total,2510.0,631.0,0.0064,(20.0/3141.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4392141,0.9841270,189.0
max f2,0.4172890,0.9898798,196.0
max f0point5,0.4620772,0.9852611,185.0
max accuracy,0.4392141,0.9936326,189.0
max precision,0.9999574,1.0,0.0
max recall,0.2375204,1.0,259.0
max specificity,0.9999574,1.0,0.0
max absolute_mcc,0.4392141,0.9801465,189.0
max min_per_class_accuracy,0.4195840,0.9924363,195.0
max mean_per_class_accuracy,0.4172890,0.9936344,196.0


Gains/Lift Table: Avg response rate: 20.03 %, avg score: 20.58 %



...

,0,1,Error,Rate
0,2504.0,8.0,0.0032,(8.0/2512.0)
1,10.0,619.0,0.0159,(10.0/629.0)
Total,2514.0,627.0,0.0057,(18.0/3141.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.5073412,0.9856688,175.0
max f2,0.3970149,0.9895669,193.0
max f0point5,0.5073412,0.9866114,175.0
max accuracy,0.5073412,0.9942693,175.0
max precision,0.9999870,1.0,0.0
max recall,0.1943388,1.0,255.0
max specificity,0.9999870,1.0,0.0
max absolute_mcc,0.5073412,0.9820894,175.0
max min_per_class_accuracy,0.4323423,0.9920509,189.0
max mean_per_class_accuracy,0.3970149,0.9934353,193.0


Gains/Lift Table: Avg response rate: 20.03 %, avg score: 20.35 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0101878,1.0000000,4.9936407,4.9936407,1.0,1.0,1.0,1.0,0.0508744,0.0508744,399.3640700,399.3640700
,2,0.0200573,1.0000000,4.9936407,4.9936407,1.0,1.0000000,1.0,1.0000000,0.0492846,0.1001590,399.3640700,399.3640700
,3,0.0302451,1.0000000,4.9936407,4.9936407,1.0,1.0000000,1.0,1.0000000,0.0508744,0.1510334,399.3640700,399.3640700
,4,0.0401146,1.0000000,4.9936407,4.9936407,1.0,1.0000000,1.0,1.0000000,0.0492846,0.2003180,399.3640700,399.3640700
,5,0.0503025,0.9999992,4.9936407,4.9936407,1.0,0.9999997,1.0,0.9999999,0.0508744,0.2511924,399.3640700,399.3640700
,6,0.1002865,0.9984403,4.9936407,4.9936407,1.0,0.9997898,1.0,0.9998952,0.2496025,0.5007949,399.3640700,399.3640700
,7,0.1502706,0.9493130,4.9936407,4.9936407,1.0,0.9845580,1.0,0.9947936,0.2496025,0.7503975,399.3640700,399.3640700
,8,0.2002547,0.4914982,4.6755744,4.9142505,0.9363057,0.7637601,0.9841017,0.9371271,0.2337043,0.9841017,367.5574413,391.4250545
,9,0.3002229,0.0184439,0.1590331,3.3308590,0.0318471,0.1535377,0.6670201,0.6762076,0.0158983,1.0,-84.0966857,233.0858961
,10,0.4001910,0.0002510,0.0,2.4988067,0.0,0.0044582,0.5003978,0.5084038,0.0,1.0,-100.0,149.8806683


In [17]:
# Retreive test set AUC
print(glm_perf1.auc())
print(glm_perf2.auc())

0.9996310238676293
0.9996810223486882


In [18]:
# Compare test AUC to the training AUC and validation AUC
print(glm_fit2.auc(train=True))
print(glm_fit2.auc(valid=True))

0.9999582598942477
0.9998349324067821


### Random Forrest

In [19]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [20]:
# Initialize the RF estimator:

rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1)

In [21]:
rf_fit1.train(x=x, y=y, training_frame=train)

drf Model Build progress: |█████████████████████████████████████████████████| 100%


In [22]:
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit2', ntrees=100, seed=1)
rf_fit2.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [23]:
rf_perf1 = rf_fit1.model_performance(test)
rf_perf2 = rf_fit2.model_performance(test)

In [24]:
# Retreive test set AUC
print(rf_perf1.auc())
print(rf_perf2.auc())

0.9839413106437274
0.9872896266442539


In [25]:
#Cross Validate Performance
rf_fit3 = H2ORandomForestEstimator(model_id='rf_fit3', seed=1, nfolds=5)
rf_fit3.train(x=x, y=y, training_frame=data)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [26]:
print(rf_fit3.auc(xval=True))

0.9870361536495137


### Gradient Boosting

In [27]:
# Import H2O GBM:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [28]:
# GBM hyperparameters
gbm_params1 = {'learn_rate': [0.01, 0.1], 
                'max_depth': [3, 5, 9],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.2, 0.5, 1.0]}

In [29]:
# Initialize and train the GBM estimator:
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,grid_id='gbm_grid1', hyper_params=gbm_params1)
gbm_grid1.train(x=x, y=y, training_frame=train, validation_frame=valid, ntrees=100, seed=1)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [30]:
gbm_fit2 = H2OGradientBoostingEstimator(model_id='gbm_fit2', ntrees=500, seed=1)
gbm_fit2.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [31]:
# Now let's use early stopping to find optimal ntrees

gbm_fit3 = H2OGradientBoostingEstimator(model_id='gbm_fit3', 
                                        ntrees=500, 
                                        score_tree_interval=5,     #used for early stopping
                                        stopping_rounds=3,         #used for early stopping
                                        stopping_metric='AUC',     #used for early stopping
                                        stopping_tolerance=0.0005, #used for early stopping
                                        seed=1)

# The use of a validation_frame is recommended with using early stopping
gbm_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [32]:
gbm_gridperf = gbm_grid1.get_grid(sort_by='auc', decreasing=True)
# Grab the model_id for the top GBM model, chosen by validation AUC
best_gbm_model = gbm_gridperf.models[0]

gbm_perf1 = best_gbm_model.model_performance(test)
gbm_perf2 = gbm_fit2.model_performance(test)
gbm_perf3 = gbm_fit3.model_performance(test)

In [33]:
# Retreive test set AUC
print(gbm_perf1.auc())
print(gbm_perf2.auc())
print(gbm_perf3.auc())

0.9990171184672871
0.9983487843407289
0.9985089060585501


## Deep Learning

In [34]:
# Import H2O DL:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [35]:
# Initialize and train the DL estimator:

dl_fit1 = H2ODeepLearningEstimator(model_id='dl_fit1', seed=1)
dl_fit1.train(x=x, y=y, training_frame=train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [36]:
dl_fit2 = H2ODeepLearningEstimator(model_id='dl_fit2', 
                                   epochs=20, 
                                   hidden=[10,10], 
                                   stopping_rounds=0,  #disable early stopping
                                   seed=1)
dl_fit2.train(x=x, y=y, training_frame=train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [37]:
dl_fit3 = H2ODeepLearningEstimator(model_id='dl_fit3', 
                                   epochs=20, 
                                   hidden=[10,10],
                                   score_interval=1,          #used for early stopping
                                   stopping_rounds=3,         #used for early stopping
                                   stopping_metric='AUC',     #used for early stopping
                                   stopping_tolerance=0.0005, #used for early stopping
                                   seed=1)
dl_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [41]:
# Import H2O DL:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [42]:
# DL hyperparameters
activation_opt = ["Rectifier", "RectifierWithDropout", "Maxout", "MaxoutWithDropout"]
l1_opt = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1]
l2_opt = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1]
dl_params = {'activation': activation_opt, 'l1': l1_opt, 'l2': l2_opt}

# Search criteria
search_criteria = {'strategy': 'RandomDiscrete', 'max_runtime_secs': 120, 'seed':1}

In [43]:
#dl_grid = H2OGridSearch(model=H2ODeepLearningEstimator,
#                        grid_id='dl_grid1',
#                        hyper_params=dl_params,
#                        search_criteria=search_criteria)

#dl_grid.train(x=x, y=y,
#              training_frame=train, 
#              validation_frame=valid, 
#              hidden=[10,10],
#              hyper_params=dl_params,
#              search_criteria=search_criteria)

#dl_gridperf = dl_grid.get_grid(sort_by='auc', decreasing=True)

In [44]:
# Grab the model_id for the top GBM model, chosen by validation AUC
#best_dl_model = dl_gridperf.models[0]

# Now let's evaluate the model performance on a test set
# so we get an honest estimate of top model performance

#dl_perf = best_gbm_model.model_performance(test)
#print dl_perf.auc()

In [45]:
dl_perf1 = dl_fit1.model_performance(test)
dl_perf2 = dl_fit2.model_performance(test)
dl_perf3 = dl_fit3.model_performance(test)

In [46]:
# Retreive test set AUC
print(dl_perf1.auc())
print(dl_perf2.auc())
print(dl_perf3.auc())

0.9986994066003058
0.9991022424635201
0.998821554788209


In [49]:
# Grab the model_id for the top GBM model, chosen by validation AUC
#best_dl_model = dl_gridperf.models[0]

# Now let's evaluate the model performance on a test set
# so we get an honest estimate of top model performance

#dl_perf = best_gbm_model.model_performance(test)
#print(dl_perf.auc())